In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost -q
!pip install kaggle -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 20.4 MB/s eta 0:00:00


In [1]:
# from google.colab import files
# files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c child-mind-institute-problematic-internet-use

100% 6.19G/6.21G [00:28<00:00, 238MB/s]
100% 6.21G/6.21G [00:29<00:00, 230MB/s]


In [ ]:
!mkdir -p ./data/child-mind-institute
!unzip child-mind-institute-problematic-internet-use.zip -d ./data/child-mind-institute
!ls ./data/child-mind-institute

Archive:  child-mind-institute-problematic-internet-use.zip
  inflating: ./data/child-mind-institute/data_dictionary.csv  
  inflating: ./data/child-mind-institute/sample_submission.csv  
  inflating: ./data/child-mind-institute/series_test.parquet/id=00115b9f/part-0.parquet  
  inflating: ./data/child-mind-institute/series_test.parquet/id=001f3379/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=00115b9f/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=001f3379/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=00f332d1/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=01085eb3/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=012cadd8/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=012e3869/part-0.parquet  
  inflating: ./data/child-mind-institute/series_train.parquet/id=029a19c9/part-0.parqu

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone, BaseEstimator, RegressorMixin
from sklearn.metrics import cohen_kappa_score, accuracy_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# import random
# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True
# seed_everything(2024)

In [ ]:
train_df = pd.read_csv('./data/child-mind-institute/train.csv')
test_df = pd.read_csv('./data/child-mind-institute/test.csv')
sample = pd.read_csv('./data/child-mind-institute/sample_submission.csv')

train_df.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class CFG:
    SEED_LIST = [42]
    N_SPLITS = 5

    # lgb_params = {
    # 'objective'       : 'l2',
    # 'verbosity'       : -1,
    # # 'n_iter'          : 200,
    # 'lambda_l1'       : 0.005116829730239727,
    # 'lambda_l2'       : 0.0011520776712645852,
    # 'learning_rate'   : 0.02376367323636638,
    # 'max_depth'       : 5,
    # 'num_leaves'      : 207,
    # 'colsample_bytree': 0.7759862336963801,
    # 'colsample_bynode': 0.5110355095943208,
    # 'bagging_fraction': 0.5485770314992224,
    # 'bagging_freq'    : 7,
    # 'min_data_in_leaf': 78,
    # }

    lgb_params = {
    'objective'       : 'l2',
    'verbosity'       : -1,
    # 'n_iter'          : 200,
    'learning_rate'   : 0.025,
    'max_depth'       : 5,
    'num_leaves'      : 207,
    'min_data_in_leaf': 150,
    'lambda_l2'       : 5,
    'lambda_l1'       : 2,
    'bagging_freq'    : 7,
    }

In [ ]:
cat_cols = train_df.select_dtypes(include=['object']).columns
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
cat_cols

Index(['id', 'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'PCIAT-Season', 'SDS-Season', 'PreInt_EduHx-Season'], dtype='object')

In [ ]:
train_df.shape

(3960, 82)

In [ ]:
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)
columns_not_in_test = sorted(list(train_cols - test_cols))

In [ ]:
train_with_sii = train_df[train_df['sii'].notna()][columns_not_in_test]
train_with_sii[train_with_sii.isna().any(axis=1)].head().style.applymap(
    lambda x: 'background-color: #FFC0CB' if pd.isna(x) else ''
)

,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,PCIAT-Season,sii
24,2.000000,2.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,nan,2.000000,30.000000,Summer,0.000000
93,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,Fall,0.000000
104,5.000000,2.000000,4.000000,2.000000,nan,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,45.000000,Fall,1.000000
141,1.000000,2.000000,4.000000,2.000000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,3.000000,0.000000,nan,0.000000,2.000000,0.000000,26.000000,Winter,0.000000
142,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,3.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,nan,1.000000,26.000000,Spring,0.000000


In [ ]:
PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]
recalc_total_score = train_with_sii[PCIAT_cols].sum(
    axis=1, skipna=True
)
(recalc_total_score == train_with_sii['PCIAT-PCIAT_Total']).all()

True

In [ ]:
PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]
recalc_total_score = train_with_sii[PCIAT_cols].sum(
    axis=1, skipna=True
)

In [ ]:
def recalculate_sii(row):
    if pd.isna(row['PCIAT-PCIAT_Total']):
        return np.nan
    max_possible = row['PCIAT-PCIAT_Total'] + row[PCIAT_cols].isna().sum() * 5
    if row['PCIAT-PCIAT_Total'] <= 30 and max_possible <= 30:
        return 0
    elif 31 <= row['PCIAT-PCIAT_Total'] <= 49 and max_possible <= 49:
        return 1
    elif 50 <= row['PCIAT-PCIAT_Total'] <= 79 and max_possible <= 79:
        return 2
    elif row['PCIAT-PCIAT_Total'] >= 80 and max_possible >= 80:
        return 3
    return np.nan

train_df['recalc_sii'] = train_df.apply(recalculate_sii, axis=1)

In [ ]:
mismatch_rows = train_df[
    (train_df['recalc_sii'] != train_df['sii']) & train_df['sii'].notna()
]

mismatch_rows[PCIAT_cols + [
    'PCIAT-PCIAT_Total', 'sii', 'recalc_sii'
]].style.applymap(
    lambda x: 'background-color: #FFC0CB' if pd.isna(x) else ''
)

,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,sii,recalc_sii
24,2.000000,2.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,nan,2.000000,30.000000,0.000000,nan
93,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,0.000000,nan
104,5.000000,2.000000,4.000000,2.000000,nan,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,45.000000,1.000000,nan
141,1.000000,2.000000,4.000000,2.000000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,3.000000,0.000000,nan,0.000000,2.000000,0.000000,26.000000,0.000000,nan
142,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,3.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,nan,1.000000,26.000000,0.000000,nan
270,3.000000,3.000000,4.000000,2.000000,4.000000,2.000000,1.000000,3.000000,2.000000,2.000000,4.000000,0.000000,2.000000,1.000000,4.000000,nan,2.000000,3.000000,4.000000,2.000000,48.000000,1.000000,nan
368,2.000000,3.000000,4.000000,2.000000,5.000000,1.000000,2.000000,nan,nan,nan,2.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,nan,nan,31.000000,1.000000,nan
592,3.000000,0.000000,3.000000,0.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000,0.000000,1.000000,nan,nan,1.000000,2.000000,1.000000,1.000000,0.000000,21.000000,0.000000,nan
724,3.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,nan,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,3.000000,0.000000,1.000000,29.000000,0.000000,nan
877,5.000000,5.000000,5.000000,4.000000,5.000000,0.000000,5.000000,5.000000,5.000000,5.000000,4.000000,nan,4.000000,5.000000,5.000000,1.000000,5.000000,0.000000,5.000000,5.000000,78.000000,2.000000,nan


In [ ]:
mismatch_rows

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,recalc_sii
24,011e71c3,Summer,15,0,Fall,70.0,Summer,21.177784,62.00,115.8,NaN,61.0,58.0,144.0,NaN,NaN,NaN,NaN,Summer,0.0,0.0,21.4,1.0,20.9,1.0,5.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,Fall,4.0,8.72618,21.1801,1289.740,2708.46,20.5717,79.6375,14.5659,6.61421,36.16250,2.0,35.9062,23.15960,70.9113,35.1487,56.4779,Summer,3.530,NaN,NaN,Summer,2.0,2.0,3.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,NaN,2.0,30.0,Summer,34.0,49.0,Summer,1.0,0.0,NaN
93,053d7d31,Spring,15,1,NaN,NaN,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Spring,2.0,0.0,NaN
104,05bc318e,Spring,11,0,Fall,68.0,Fall,20.095643,59.50,101.2,NaN,69.0,71.0,116.0,Fall,5.0,8.0,56.0,Fall,18.0,1.0,19.2,2.0,21.2,2.0,7.0,0.0,9.0,1.0,9.0,1.0,10.0,1.0,Fall,3.0,6.83747,20.0978,1230.280,2091.48,18.9913,73.3040,14.5578,5.54001,27.89600,3.0,34.1484,20.16430,66.4666,32.2686,53.1397,NaN,NaN,Spring,2.280,Fall,5.0,2.0,4.0,2.0,NaN,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,45.0,Fall,41.0,58.0,Spring,2.0,1.0,NaN
141,08159c5e,Winter,17,1,Fall,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,1.520,NaN,NaN,Winter,1.0,2.0,4.0,2.0,2.0,2.0,1.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,NaN,0.0,2.0,0.0,26.0,Winter,30.0,43.0,Winter,2.0,0.0,NaN
142,082df556,Winter,16,0,Summer,45.0,Spring,16.774971,70.50,118.6,NaN,64.0,64.0,115.0,NaN,NaN,NaN,NaN,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,3.0,7.00658,16.7768,1562.290,2655.89,34.2329,108.6670,15.3717,1.40506,9.93274,1.0,46.2480,28.18640,101.6610,46.4142,80.4809,Winter,2.040,NaN,NaN,Spring,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,26.0,Winter,32.0,46.0,Winter,0.0,0.0,NaN
270,0fbb3ca5,Winter,15,0,Spring,55.0,Spring,19.024041,70.00,132.6,NaN,76.0,80.0,112.0,NaN,NaN,NaN,NaN,Spring,15.0,0.0,18.6,1.0,19.3,1.0,4.0,0.0,11.0,1.0,12.0,1.0,7.0,0.0,Summer,3.0,6.60843,21.8958,1580.060,2686.10,32.3693,110.5600,15.8638,6.03206,42.03970,2.0,47.9234,30.26760,103.9520,53.2668,80.2927,Winter,2.160,NaN,NaN,Spring,3.0,3.0,4.0,2.0,4.0,2.0,1.0,3.0,2.0,2.0,4.0,0.0,2.0,1.0,4.0,NaN,2.0,3.0,4.0,2.0,48.0,Spring,34.0,49.0,Winter,3.0,1.0,NaN
368,1551c806,Fall,18,1,Spring,60.0,Winter,21.014782,67.50,136.2,NaN,64.0,87.0,115.0,NaN,NaN,NaN,NaN,Winter,20.0,1.0,28.5,2.0,27.9,2.0,10.0,1.0,6.0,0.0,8.0,0.0,9.0,1.0,Winter,3.0,5.61222,21.0171,1418.560,2269.69,32.1136,93.

In [ ]:
train_df['sii'].isnull().sum()

1224

In [ ]:
train_df['recalc_sii'].isnull().sum()

1241

In [ ]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)

    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname):
    ids = os.listdir(dirname)

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))

    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes

    return df

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [ ]:
%%time

ts_train = load_time_series("/content/data/child-mind-institute/series_train.parquet")
ts_test = load_time_series("/content/data/child-mind-institute/series_test.parquet")

time_series_cols = ts_train.columns.tolist()
time_series_cols.remove("id")

train_df = pd.merge(train_df, ts_train, how="left", on='id')
test_df = pd.merge(test_df, ts_test, how="left", on='id')

train_df = train_df.set_index('id')
test_df = test_df.set_index('id')

100%|██████████| 2/2 [00:00<00:00, 13.02it/s]

CPU times: user 3min 14s, sys: 21.2 s, total: 3min 36s
Wall time: 53.5 s


In [ ]:
def feature_engineering(data):

    data['bp_ratio'] = data['Physical-Systolic_BP'] / data['Physical-Diastolic_BP']
    data['height_waist_ratio'] = data['Physical-Height'] / data['Physical-Waist_Circumference']
    data['cardio_index'] = (data['Physical-HeartRate'] * data['Physical-Systolic_BP']) / 1000

    data['total_fitness_duration'] = (data['Fitness_Endurance-Time_Mins'] * 60 +
                                      data['Fitness_Endurance-Time_Sec'])

    data['endurance_efficiency'] = (data['Fitness_Endurance-Max_Stage'] /
                                    data['total_fitness_duration'].replace(0, 1))

    data['fat_muscle_ratio'] = data['BIA-BIA_Fat'] / (data['BIA-BIA_SMM'] + 1e-5)
    data['water_balance'] = data['BIA-BIA_ECW'] / (data['BIA-BIA_ICW'] + 1e-5)


    data['fat_tbw_ratio'] = data['BIA-BIA_Fat'] / data['BIA-BIA_TBW'] #
    data['fat_lst_ratio'] = data['BIA-BIA_Fat'] / data['BIA-BIA_LST'] #
    data['bmi_hr'] = data['Physical-BMI'] * data['Physical-HeartRate'] #
    # data['bmi_systolic'] = data['Physical-BMI'] * data['Physical-Systolic_BP'] #
    # data['bmi_diastolic'] = data['Physical-BMI'] * data['Physical-Diastolic_BP'] #
    # data['stamina_index'] = data['Fitness_Endurance-Time_Sec'] / (data['Fitness_Endurance-Max_Stage'] + 1e-5) #
    # data['age_normalized_endurance'] = data['Fitness_Endurance-Time_Mins'] / (data['Basic_Demos-Age'] + 1e-5) #
    # data['sds_cgas_ratio'] = data['SDS-SDS_Total_T'] / (data['CGAS-CGAS_Score'] + 1e-5) #
    # data['sds_raw_cgas_ratio'] = data['SDS-SDS_Total_Raw'] / (data['CGAS-CGAS_Score'] + 1e-5) #
    # data['ponderal_index'] = data['Physical-Weight'] / (data['Physical-Height']**3 + 1e-5) #


    data['muscle_height_ratio'] = data['BIA-BIA_SMM'] / data['Physical-Height']
    data['bmi_activity'] = data['Physical-BMI'] * data['BIA-BIA_Activity_Level_num']
    data['hr_fitness'] = data['Physical-HeartRate'] * data['Fitness_Endurance-Max_Stage']

    data['internet_hours_per_age'] = (data['PreInt_EduHx-computerinternet_hoursday'] /
                                      data['Basic_Demos-Age'])
    data['internet_activity_ratio'] = (
        data['PreInt_EduHx-computerinternet_hoursday'] /
        (data['PAQ_A-PAQ_A_Total'].fillna(0) + data['PAQ_C-PAQ_C_Total'].fillna(0) + 1)
    )

    data['age_adj_bmi'] = data['Physical-BMI'] / np.log1p(data['Basic_Demos-Age'])

    data['age_group'] = pd.cut(data['Basic_Demos-Age'],
                                bins=[0, 8, 12, 16, 22],
                                labels=['young_child', 'child', 'teen', 'young_adult'])

    data['screen_activity'] = (
        data['PreInt_EduHx-computerinternet_hoursday'] *
        (data['PAQ_A-PAQ_A_Total'].fillna(0) + data['PAQ_C-PAQ_C_Total'].fillna(0))
    )

    return data



train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

# new_feature_cols = ['bp_ratio', 'height_waist_ratio', 'cardio_index', 'total_fitness_duration', 'endurance_efficiency', 'fat_muscle_ratio', 'water_balance', 'fat_tbw_ratio', 'fat_lst_ratio',
#                     'bmi_hr', 'bmi_systolic', 'bmi_diastolic', 'stamina_index', 'age_normalized_endurance', 'sds_cgas_ratio', 'sds_raw_cgas_ratio', 'ponderal_index',
#                     'muscle_height_ratio', 'bmi_activity', 'hr_fitness', 'internet_hours_per_age', 'internet_activity_ratio', 'age_adj_bmi', 'screen_activity']

new_feature_cols = ['bp_ratio', 'height_waist_ratio', 'cardio_index', 'total_fitness_duration', 'endurance_efficiency', 'fat_muscle_ratio', 'water_balance', 'fat_tbw_ratio', 'fat_lst_ratio',
                    'bmi_hr',
                    'muscle_height_ratio', 'bmi_activity', 'hr_fitness', 'internet_hours_per_age', 'internet_activity_ratio', 'age_adj_bmi', 'screen_activity']


cat_cols = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season', 'age_group']
num_cols = ['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday']
tabular_cols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season', 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-Season', 'PreInt_EduHx-computerinternet_hoursday']
# target_col = 'sii'
target_col = "recalc_sii"

feature_cols = tabular_cols + time_series_cols + new_feature_cols
num_cols = num_cols + time_series_cols + new_feature_cols

In [ ]:
train_df = train_df.dropna(subset=[target_col])

In [ ]:
imputer = SimpleImputer(
    strategy='mean',
)

train_df[num_cols] = imputer.fit_transform(train_df[num_cols])
test_df[num_cols] = imputer.transform(test_df[num_cols])

In [ ]:
encoder = OrdinalEncoder(
    dtype=np.int32,
    handle_unknown='use_encoded_value',
    unknown_value=-1,
    encoded_missing_value=-2,
)

train_df[cat_cols] = encoder.fit_transform(train_df[cat_cols])
train_df[cat_cols] = train_df[cat_cols].astype('category')

test_df[cat_cols] = encoder.transform(test_df[cat_cols])
test_df[cat_cols] = test_df[cat_cols].astype('category')

In [ ]:
# @title train model t-test
# from scipy.stats import ttest_rel

# def TrainML(model_class, train_data, seeds=[42, 27485]):
#     # This list will store results for each seed
#     all_seed_results = []
#     seed_fold_qwks = []

#     for seed in seeds:
#         X = train_data[feature_cols]
#         y = train_data[target_col]

#         SKF = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=seed)

#         train_S = []
#         test_S = []

#         oof_non_rounded = np.zeros(len(y), dtype=float)
#         oof_rounded = np.zeros(len(y), dtype=int)

#         # List to store models from each fold
#         models = []

#         # Train and validate for each fold
#         for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc=f"Training Folds [seed={seed}]", total=CFG.N_SPLITS)):
#             X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#             y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#             model = clone(model_class)
#             model.fit(X_train, y_train)

#             models.append(model)

#             y_train_pred = model.predict(X_train)
#             y_val_pred = model.predict(X_val)

#             oof_non_rounded[test_idx] = y_val_pred
#             y_val_pred_rounded = y_val_pred.round(0).astype(int)
#             oof_rounded[test_idx] = y_val_pred_rounded

#             train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#             val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#             train_S.append(train_kappa)
#             test_S.append(val_kappa)

#             print(f"Seed {seed} - Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
#             # clear_output(wait=True)

#         mean_train_qwk = np.mean(train_S)
#         mean_val_qwk = np.mean(test_S)

#         print(f"Seed {seed}: Mean Train QWK --> {mean_train_qwk:.4f}")
#         print(f"Seed {seed}: Mean Validation QWK ---> {mean_val_qwk:.4f}")

#         KappaOPtimizer = minimize(
#             evaluate_predictions,
#             x0=[0.5, 1.5, 2.5],
#             args=(y, oof_non_rounded),
#             method='Nelder-Mead'
#         )
#         assert KappaOPtimizer.success, "Optimization did not converge."

#         oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
#         tKappa = quadratic_weighted_kappa(y, oof_tuned)

#         print(f"Seed {seed} ----> || Optimized QWK SCORE :: {tKappa:.3f}")

#         # Store the results for this seed
#         all_seed_results.append({
#             'seed': seed,
#             'mean_train_qwk': mean_train_qwk,
#             'mean_val_qwk': mean_val_qwk,
#             'optimized_thresholds': KappaOPtimizer.x,
#             'optimized_qwk': tKappa,
#             'models': models
#         })


#         seed_fold_qwks.append(test_S)

#     # Print all seed results at the end
#     print("\nSummary of all seeds:")
#     for res in all_seed_results:
#         print(f"Seed {res['seed']}: "
#               f"Mean Train QWK = {res['mean_train_qwk']:.4f}, "
#               f"Mean Validation QWK = {res['mean_val_qwk']:.4f}, "
#               f"Optimized QWK = {res['optimized_qwk']:.3f}")


#     num_seeds = len(seeds)
#     for i in range(num_seeds):
#         for j in range(i+1, num_seeds):
#             t_stat, p_val = ttest_rel(seed_fold_qwks[i], seed_fold_qwks[j])
#             print(f"T-test between seed {all_seed_results[i]['seed']} and {all_seed_results[j]['seed']}:")
#             print(f"  T-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")

#     return all_seed_results


In [ ]:
from scipy.stats import ttest_rel
import random

def TrainML(model_class, train_data, max_seeds=500):
    # This list will store results for seed 42 and accepted seeds
    all_seed_results = []
    accepted_seeds = []
    seed_fold_qwks = {}

    # Static seed for comparison
    fixed_seed = 42

    # Generate results for the fixed seed first
    seed_results_42 = run_model_with_seed(fixed_seed, model_class, train_data)
    seed_fold_qwks[fixed_seed] = seed_results_42['fold_qwks']
    all_seed_results.append(seed_results_42)

    print(f"Fixed seed ({fixed_seed}) results calculated.")

    while len(accepted_seeds) < max_seeds:
        # Generate a random seed
        new_seed = random.randint(1, 1e6)

        if new_seed in seed_fold_qwks:
            continue  # Avoid duplicate seed processing

        # Run model with the new seed
        new_seed_results = run_model_with_seed(new_seed, model_class, train_data)
        seed_fold_qwks[new_seed] = new_seed_results['fold_qwks']

        # Perform t-test between the fixed seed and the new seed
        t_stat, p_val = ttest_rel(seed_fold_qwks[fixed_seed], seed_fold_qwks[new_seed])

        print(f"Seed {new_seed} compared with fixed seed {fixed_seed}: T-stat = {t_stat:.4f}, P-value = {p_val:.4f}")

        # Check p-value threshold
        if p_val < 0.05:
            accepted_seeds.append(new_seed)
            all_seed_results.append(new_seed_results)
            print(f"Seed {new_seed} added to the accepted seeds list with T-stat = {t_stat:.4f}, P-value = {p_val:.4f}")

    print("\nFinal accepted seeds:", accepted_seeds)
    return all_seed_results, accepted_seeds

def run_model_with_seed(seed, model_class, train_data):
    X = train_data[feature_cols]
    y = train_data[target_col]

    SKF = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=seed)

    fold_qwks = []
    oof_non_rounded = np.zeros(len(y), dtype=float)

    # Train and validate for each fold
    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        y_val_pred_rounded = y_val_pred.round(0).astype(int)

        oof_non_rounded[test_idx] = y_val_pred
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        fold_qwks.append(val_kappa)

    mean_val_qwk = np.mean(fold_qwks)

    KappaOptimizer = minimize(
        evaluate_predictions,
        x0=[0.5, 1.5, 2.5],
        args=(y, oof_non_rounded),
        method='Nelder-Mead'
    )
    assert KappaOptimizer.success, "Optimization did not converge."

    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    return {
        'seed': seed,
        'fold_qwks': fold_qwks,
        'mean_val_qwk': mean_val_qwk,
        'optimized_qwk': tKappa,
        'optimized_thresholds': KappaOptimizer.x
    }


In [ ]:
def MakePredictions(test_data, models, thresholds):
    # Get predictions from each fold's model

    test_data = test_data[feature_cols]

    preds_per_model = []
    for model in models:
        preds = model.predict(test_data)
        preds_per_model.append(preds)

    # Average predictions from all models (ensemble)
    mean_preds = np.mean(preds_per_model, axis=0)

    # Apply the optimized thresholds to convert continuous predictions to discrete classes
    final_predictions = threshold_Rounder(mean_preds, thresholds)

    return final_predictions


In [ ]:
lgbm = LGBMRegressor(**CFG.lgb_params, random_state=42, verbose=2, n_estimators=200)


results, accepted_seeds = TrainML(lgbm, train_df)

# all_seed_results = TrainML(lgbm, train_df)

# # Since we are using only one seed, `all_seed_results` has one element
# result1 = all_seed_results[0]
# result2 = all_seed_results[1]
# result3 = all_seed_results[2]
# result4 = all_seed_results[3]

# # Extract the thresholds and models from the dictionary
# thresholds1 = result1['optimized_thresholds']
# models1 = result1['models']
# thresholds2 = result2['optimized_thresholds']
# models2 = result2['models']
# thresholds3 = result3['optimized_thresholds']
# models3 = result3['models']
# thresholds4 = result4['optimized_thresholds']
# models4 = result4['models']


# # Now you can pass these parameters to your MakePredictions function
# test_preds1 = MakePredictions(test_df, models1, thresholds1)
# test_preds2 = MakePredictions(test_df, models2, thresholds2)
# test_preds3 = MakePredictions(test_df, models3, thresholds3)
# test_preds4 = MakePredictions(test_df, models4, thresholds4)

# final_preds = (test_preds1 + test_preds2 + test_preds3 + test_preds4) / 4
# final_preds = final_preds.round(0).astype(int)

# print("Test Predictions:", final_preds)


Fixed seed (42) results calculated.
Seed 209927 compared with fixed seed 42: T-stat = 0.4973, P-value = 0.6451
Seed 930400 compared with fixed seed 42: T-stat = 0.2391, P-value = 0.8228
Seed 758661 compared with fixed seed 42: T-stat = 0.7590, P-value = 0.4901
Seed 430010 compared with fixed seed 42: T-stat = 0.1222, P-value = 0.9087
Seed 794000 compared with fixed seed 42: T-stat = 0.3121, P-value = 0.7705
Seed 751425 compared with fixed seed 42: T-stat = 1.0226, P-value = 0.3643
Seed 795477 compared with fixed seed 42: T-stat = 0.3298, P-value = 0.7581
Seed 278105 compared with fixed seed 42: T-stat = -0.0495, P-value = 0.9629
Seed 559081 compared with fixed seed 42: T-stat = 0.2340, P-value = 0.8265
Seed 257076 compared with fixed seed 42: T-stat = 0.5293, P-value = 0.6246
Seed 667050 compared with fixed seed 42: T-stat = 0.2315, P-value = 0.8283
Seed 852054 compared with fixed seed 42: T-stat = 0.4916, P-value = 0.6487
Seed 770958 compared with fixed seed 42: T-stat = 0.7265, P-val

KeyboardInterrupt: 

In [ ]:
train_df.head()

In [ ]:
# default: Seed 42: Mean Train QWK = 0.4862, Mean Validation QWK = 0.3730, Optimized QWK = 0.468
# default Seed 42: Mean Train QWK = 0.4839, Mean Validation QWK = 0.3729, Optimized QWK = 0.475



# min_data_in_leaf: 200 Seed 42: Mean Train QWK = 0.4589, Mean Validation QWK = 0.3614, Optimized QWK = 0.462

# min_data_in_leaf: 180 Seed 42: Mean Train QWK = 0.4699, Mean Validation QWK = 0.3705, Optimized QWK = 0.465

# min_data_in_leaf: 150 Seed 42: Mean Train QWK = 0.4853, Mean Validation QWK = 0.3777, Optimized QWK = 0.473

# min_data_in_leaf: 120 Seed 42: Mean Train QWK = 0.5056, Mean Validation QWK = 0.3700, Optimized QWK = 0.460


# lambda_l2: 3 Seed 42: Mean Train QWK = 0.4874, Mean Validation QWK = 0.3782, Optimized QWK = 0.471
# lambda_l2: 5 Seed 42: Mean Train QWK = 0.4852, Mean Validation QWK = 0.3809, Optimized QWK = 0.471
# lambda_l2: 7 Seed 42: Mean Train QWK = 0.4843, Mean Validation QWK = 0.3784, Optimized QWK = 0.471
# lambda_l2: 10 Seed 42: Mean Train QWK = 0.4843, Mean Validation QWK = 0.3780, Optimized QWK = 0.469


# lambda_l1: 2 Seed 42: Mean Train QWK = 0.4807, Mean Validation QWK = 0.3822, Optimized QWK = 0.476





In [ ]:
submission = pd.DataFrame({
        'id': sample['id'],
        'sii': final_preds
    })

submission.to_csv('submission.csv', index=False)